In [1]:
from lightgbm import Dataset
from lightgbm import train
from pandas import DataFrame
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
train_df = read_csv('../input/spaceship-titanic/train.csv')
print(train_df.columns.tolist())

# get the labels
y = train_df['Transported'].values
train_df = train_df.drop(columns=['PassengerId', 'Transported'], inplace=False)
columns = ['CryoSleep', 'Destination', 'HomePlanet', 'VIP',]
for column in columns:
    encoder = LabelEncoder()
    train_df[column] = encoder.fit_transform(y=train_df[column])
# train_df = train_df.drop(columns=['CryoSleep', 'Destination', 'HomePlanet', 'VIP', 'Cabin', 'Name',], inplace=False)
train_df = train_df.drop(columns=['Cabin', 'Name',], inplace=False)

x = train_df.values

# split
random_state = 1
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=random_state, stratify=y)
train_data = Dataset(x, label=y, categorical_feature='auto')
test_data = Dataset(x_test, label=y_test)

['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported']


In [3]:
parameters = {
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'boosting': 'gbdt',
    'deterministic': True,
    'feature_fraction': 0.5,
    'is_unbalance': False,
    'learning_rate': 0.05,
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbosity': -2
}

model = train(parameters, train_data, valid_sets=test_data, num_boost_round=200,)

[1]	valid_0's auc: 0.804615
[2]	valid_0's auc: 0.831956
[3]	valid_0's auc: 0.843195
[4]	valid_0's auc: 0.849227
[5]	valid_0's auc: 0.852409
[6]	valid_0's auc: 0.855131
[7]	valid_0's auc: 0.853891
[8]	valid_0's auc: 0.851717
[9]	valid_0's auc: 0.854636
[10]	valid_0's auc: 0.857012
[11]	valid_0's auc: 0.856936
[12]	valid_0's auc: 0.856858
[13]	valid_0's auc: 0.856859
[14]	valid_0's auc: 0.859373
[15]	valid_0's auc: 0.859503
[16]	valid_0's auc: 0.860255
[17]	valid_0's auc: 0.86296
[18]	valid_0's auc: 0.864209
[19]	valid_0's auc: 0.863959
[20]	valid_0's auc: 0.863719
[21]	valid_0's auc: 0.863911
[22]	valid_0's auc: 0.863229
[23]	valid_0's auc: 0.86295
[24]	valid_0's auc: 0.862889
[25]	valid_0's auc: 0.862946
[26]	valid_0's auc: 0.864176
[27]	valid_0's auc: 0.864075
[28]	valid_0's auc: 0.865346
[29]	valid_0's auc: 0.864979
[30]	valid_0's auc: 0.864574
[31]	valid_0's auc: 0.865446
[32]	valid_0's auc: 0.865046
[33]	valid_0's auc: 0.86464
[34]	valid_0's auc: 0.864814
[35]	valid_0's auc: 0.8644

In [4]:
submission_df = read_csv('../input/spaceship-titanic/test.csv')
ids = submission_df['PassengerId'].values
columns = ['CryoSleep', 'Destination', 'HomePlanet', 'VIP',]
for column in columns:
    encoder = LabelEncoder()
    submission_df[column] = encoder.fit_transform(y=submission_df[column])

submission_df = submission_df.drop(columns=['PassengerId', 'Cabin', 'Name',], inplace=False, axis=1)
output_df = DataFrame({'PassengerId': ids, 'Transported': model.predict(submission_df.values) > 0.5})
output_df.to_csv('submission.csv', index=False)